In [50]:


!pip install boto3 s3fs geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 11.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [46]:

os.environ['AWS_S3_ENDPOINT'] = "minio.dive.edito.eu"
os.environ['AWS_ACCESS_KEY_ID'] = "3S2DZJT5G18N4HMWP3HQ"
os.environ['AWS_SECRET_ACCESS_KEY'] = "6xqYrn+o+RYE+6qLSG6wIzNG7ulzaOMxnZmP9bU+"
os.environ['AWS_SESSION_TOKEN'] = "eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiIzUzJEWkpUNUcxOE40SE1XUDNIUSIsImFjciI6IjAiLCJhbGxvd2VkLW9yaWdpbnMiOlsiKiJdLCJhdWQiOlsibWluaW8iLCJhY2NvdW50Il0sImF1dGhfdGltZSI6MTc1MTg3ODU0MywiYXpwIjoib255eGlhLW1pbmlvIiwiZW1haWwiOiJwemFib3Jvd3NraUBvZ2Mub3JnIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImV4cCI6MTc1MjA2MTEyNSwiZmFtaWx5X25hbWUiOiJaYWJvcm93c2tpIiwiZ2l2ZW5fbmFtZSI6IlBpb3RyIiwiZ3JvdXBzIjpbIkVESVRPX1VTRVIiXSwiaWF0IjoxNzUxOTc0NzI1LCJpc3MiOiJodHRwczovL2F1dGguZGl2ZS5lZGl0by5ldS9hdXRoL3JlYWxtcy9kYXRhbGFiIiwianRpIjoiOTU5MDc2YzItNmIyZS00NDM3LTgwN2QtZDg0YTlhMjdkNDFhIiwibmFtZSI6IlBpb3RyIFphYm9yb3dza2kiLCJwb2xpY3kiOiJzdHNvbmx5IiwicHJlZmVycmVkX3VzZXJuYW1lIjoicHphYm9yb3dza2kiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1kYXRhbGFiIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfSwibWluaW8iOnsicm9sZXMiOlsic3Rzb25seSJdfX0sInNjb3BlIjoib3BlbmlkIGVtYWlsIHByb2ZpbGUiLCJzZXNzaW9uX3N0YXRlIjoiMzkxZmU4YTYtYjUxOS00NGM4LTk1N2UtMGYzOTdjMGMzYWMzIiwic2lkIjoiMzkxZmU4YTYtYjUxOS00NGM4LTk1N2UtMGYzOTdjMGMzYWMzIiwic3ViIjoiNjNmMTcxYjktNTJiYS00NWZlLTkwYjMtN2QxNzZiNzdmOTA0IiwidHlwIjoiQmVhcmVyIn0.zJE10v-O8sdtnPbejfi-5ffqYMcytq6GyzIU8r4rXFBrviXWBauh_kaw8j7xjCOCjDArkwCVGHAtWuzxXTk6cw"
os.environ['AWS_DEFAULT_REGION'] = "waw3-1"

In [47]:
import os
bucket_name = "oidc-pzaborowski"
file_key = "obs.csv"
#initiate credentials for boto3
s3 = boto3.client('s3',
                  endpoint_url='https://'+os.environ.get('AWS_S3_ENDPOINT'),
                  aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
                  aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
                  aws_session_token=os.environ.get('AWS_SESSION_TOKEN'),
                  region_name=os.environ.get('AWS_DEFAULT_REGION'))
response = s3.get_object(Bucket=bucket_name, Key=file_key)

In [48]:
response = s3.get_object(Bucket=bucket_name, Key=file_key)
csv = response['Body'].read().decode('utf-8')
csv

'id,stn_id,datetime,value,lat,long\n371,35,"2001-10-30T14:24:55Z",89.9,45,-75\n377,35,"2002-10-30T18:31:38Z",93.9,45,-75\n238,2147,"2007-10-30T08:57:29Z",103.5,43,-79\n297,2147,"2003-10-30T07:37:29Z",93.5,43,-79\n964,604,"2000-10-30T18:24:39Z",99.9,49,-122\n'

In [54]:
import geopandas as gpd
import pandas as pd
from io import StringIO

# Read CSV string into DataFrame
df = pd.read_csv(StringIO(csv))

# Convert to GeoDataFrame using lat/long
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['long'], df['lat']),
    crs="EPSG:4326"
)

# Save as GeoParquet
gdf.to_parquet('obs.parquet', index=False)

In [55]:

#upload geoparquet to s3
s3.upload_file('obs.parquet', bucket_name, 'obs.parquet')

#clean up
os.remove('obs.parquet')

In [67]:
# Source url is required
source = "s3://oidc-pzaborowski/obs.parquet"
import pyarrow
import pyarrow.compute as pc
import pyarrow.dataset
import s3fs

# Manage AWS S3 sources
if source.startswith('s3'):
    source = source.split('://', 1)[1]
    fs = s3fs.S3FileSystem(
        client_kwargs={'endpoint_url': 'https://'+os.environ["AWS_S3_ENDPOINT"]},
        key = os.environ["AWS_ACCESS_KEY_ID"], 
        secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
        token = os.environ["AWS_SESSION_TOKEN"],
        default_cache_type='none')
else:
    fs = None

ds = pyarrow.dataset.dataset(source, filesystem=fs)
ds

In [68]:
def get_fields():
        """
        Get provider field information (names, types)

        :returns: dict of fields
        """

        fields = dict()

        for field_name, field_type in zip(ds.schema.names,
                                          ds.schema.types):
            # Geometry is managed as a special case by pygeoapi
            if field_name == 'geometry':
                continue

            field_type = str(field_type)
            converted_type = None
            converted_format = None
            if field_type.startswith(('int', 'uint')):
                converted_type = 'integer'
                converted_format = field_type
            elif field_type == 'double' or field_type.startswith('float'):
                converted_type = 'number'
                converted_format = field_type
            elif field_type == 'string':
                converted_type = 'string'
            elif field_type == 'bool':
                converted_type = 'boolean'
            elif field_type.startswith('timestamp'):
                converted_type = 'string'
                converted_format = 'date-time'
            else:
                LOGGER.error(f'Unsupported field type {field_type}')

            if converted_format is None:
                fields[field_name] = {'type': converted_type}
            else:
                fields[field_name] = {
                    'type': converted_type,
                    'format': converted_format,
                }

        return fields
fields = get_fields()
fields

{'id': {'type': 'integer', 'format': 'int64'},
 'stn_id': {'type': 'integer', 'format': 'int64'},
 'datetime': {'type': 'string'},
 'value': {'type': 'number', 'format': 'double'},
 'lat': {'type': 'integer', 'format': 'int64'},
 'long': {'type': 'integer', 'format': 'int64'}}